In [1]:
"""
Approach:
1 - load all numbers from all reports
2 - line-by-line, digit-by-digit populating of JSON file
3 - get it to work, refactor later as needed.
"""

import os
import json
import pandas as pd
import arcpy
arcpy.env.workspace = r'I:\Projects\Darren\PPA_V2_GIS\PPA_V2.gdb'

import ppa_input_params as params
from accessibility_calcs import get_acc_data
from mix_index_for_project import get_mix_idx
from landuse_buff_calcs import LandUseBuffCalcs

# load json template
in_json = r"C:\Users\dconly\GitRepos\PPA3\testing\json_output\SACOG_ReduceVMT_template.json"
with open(in_json, "r") as j_in:
    json_loaded = json.load(j_in)


# specify project line feature class and attributes
project_fc = r'I:\Projects\Darren\PPA_V2_GIS\PPA_V2.gdb\Polylines'

ptype = params.ptype_arterial
data_years = [2016, 2040]
geo_project = "Project"
geo_ctype = "Community Type"
geo_region = "Region"


# csv table of aggregate values
aggval_csv = r"C:\Users\dconly\GitRepos\PPA2\ppa\Input_Template\CSV\Agg_ppa_vals04222020_1017.csv"

# re-used json keys
k_charts = "charts"
k_features = "features" # remember, this is a list of dicts
k_attrs = "attributes"
k_year = "year"
k_type = "type"

geographies = [geo_project, geo_ctype, geo_region]


In [2]:
# identify project community type
def get_proj_ctype(in_project_fc, commtypes_fc):
    '''Get project community type, based on which community type has most spatial overlap with project'''
    temp_intersect_fc = os.path.join(arcpy.env.scratchGDB, 'temp_intersect_fc') # don't use "memory", use scratch GDB
    arcpy.Intersect_analysis([in_project_fc, commtypes_fc], temp_intersect_fc, "ALL", 
                             0, "LINE")
    
    # debugging messages to find out why ctype tagging intermittently fails
    intersect_cnt = int(arcpy.GetCount_management(temp_intersect_fc)[0])
    in_project_cnt = int(arcpy.GetCount_management(in_project_fc)[0])
    arcpy.AddMessage("project line feature count: {}".format(in_project_cnt))
    arcpy.AddMessage("Project segments after intersecting with comm types: {}".format(intersect_cnt))
    
    len_field = 'SHAPE@LENGTH'
    fields = ['OBJECTID', len_field, params.col_ctype]
    ctype_dist_dict = {}
    
    with arcpy.da.SearchCursor(temp_intersect_fc, fields) as cur:
        for row in cur:
            ctype = row[fields.index(params.col_ctype)]
            seg_len = row[fields.index(len_field)]
        
            if ctype_dist_dict.get(ctype) is None:
                ctype_dist_dict[ctype] = seg_len
            else:
                ctype_dist_dict[ctype] += seg_len
    try:
        maxval = max([v for k, v in ctype_dist_dict.items()])
        proj_ctype = [k for k, v in ctype_dist_dict.items() if v == maxval][0]

        return proj_ctype
    except:
        raise ValueError("ERROR: No Community Type identified for project. \n{} project line features." \
                         " {} features in intersect layer.".format(in_project_cnt, in_project_cnt))
        
project_commtype = get_proj_ctype(project_fc, params.comm_types_fc)

In [3]:
# get all applicable comm type and region aggregate values
df_agg = pd.read_csv(aggval_csv)
metname_col = "metric_name"
df_agg = df_agg.rename(columns={'Unnamed: 0':metname_col, 'REGION': geo_region})
df_agg.head()

,metric_name,Ag,Arterials & Suburban Corridors,Developing,Established Communities,Rural & Small Town Main Street,Rural Residential,Small-Town Established Communities,Urban core,Region,year
0,WALKDESTSalljob,578.934547,5337.598083,945.143105,3208.991897,2121.642656,469.990251,1364.000857,64913.702561,3842.410878,2016
1,BIKEDESTSalljob,4181.014226,58753.268958,6511.420259,37113.506493,8340.183800,4359.477548,7127.598228,213458.712809,37354.023709,2016
2,AUTODESTSalljob,153980.014225,581272.181393,308544.671363,510067.786983,150264.423101,180455.785539,115470.053128,775596.880729,475047.245696,2016
3,TRANDESTSalljob,1998.225747,39399.817028,3391.266012,20656.833790,5562.764841,1608.656246,3942.737866,163632.287795,22213.193547,2016
4,WALKDESTSedu,1.083223,6.116137,1.678553,4.806067,3.959830,0.874987,2.835966,15.148187,4.638241,2016


In [30]:
# make dict of region and community type values for specified metric and community type
def make_aggval_dict(aggval_df, metric_cols, proj_ctype):
    aggdatadf_proj = aggval_df.loc[aggval_df[metname_col].isin(metric_cols)] \
                    [[metname_col, project_commtype, geo_region, k_year]]

    agg_dict_list = aggdatadf_proj.to_dict(orient='records')
    aggval_dict = {d[metname_col]:{project_commtype:d[proj_ctype], 
                      geo_region:d[geo_region],
                      k_year:d[k_year]} for d in agg_dict_list}

    return aggval_dict

In [4]:
# calc land use buffer values 
buffdist_ft = 2640
value_fields = ['EMPTOT', 'DU_TOT']

out_data = {}
for data_year in data_years:
    in_pcl_pt_fc = params.parcel_pt_fc_yr(data_year)
    year_dict = LandUseBuffCalcs(in_pcl_pt_fc, project_fc, ptype, value_fields, buffdist_ft).point_sum()
    out_data[data_year] = year_dict
    

# update applicable field values in JSON template
for i, year in enumerate(data_years):
    jobs = out_data[year]['EMPTOT']
    du = out_data[year]['DU_TOT']
    json_loaded[k_charts]["Jobs and Dwelling"][k_features][i][k_attrs]['year'] = year
    json_loaded[k_charts]["Jobs and Dwelling"][k_features][i][k_attrs]['jobs'] = jobs
    json_loaded[k_charts]["Jobs and Dwelling"][k_features][i][k_attrs]['dwellingUnits'] = du

# print(json.dumps(json_loaded, indent=4))

In [31]:
# calc accessibility numbers and update JSON chart with it

fc_accessibility_data = params.accdata_fc

# project level dict of accessibility script outputs
dict_data = get_acc_data(project_fc, fc_accessibility_data, ptype) 

# lookup dict between names of data points in raw output and names in JSON file
dest_type = 'poi2' # destination type as specified in raw data output
acc_metrics = {f'WALKDESTS{dest_type}':'30 Min Walk', f'BIKEDESTS{dest_type}':'30 Min Biking', 
                f'AUTODESTS{dest_type}':'15 Min Drive', f'TRANDESTS{dest_type}':'45 Min Transit'}
accmetrics_keys = list(acc_metrics.keys())

# trimmed down output, only containing the accessibility metrics needed for this chart 
# instead of all accessibility metrics
dict_data2 = {k:dict_data[k] for k in acc_metrics.keys()}

# make dict of regional and comm type values
aggval_dict = make_aggval_dict(df_agg, accmetrics_keys, project_commtype)

for i, k in enumerate(list(acc_metrics.keys())):
    # update value for "type" (mode of tranpsortation)
    json_loaded[k_charts]["Base Year Service Accessibility"][k_features][i] \
        [k_attrs][k_type] = acc_metrics[k]
    
    # update value for mode's access from project
    json_loaded[k_charts]["Base Year Service Accessibility"][k_features][i] \
        [k_attrs][geo_project] = dict_data2[k] 
    
    # update value for mode's access avg for project comm type
    json_loaded[k_charts]["Base Year Service Accessibility"][k_features][i] \
        [k_attrs][geo_ctype] = aggval_dict[k][project_commtype] 
    
    # update value for mode's access avg for region
    json_loaded[k_charts]["Base Year Service Accessibility"][k_features][i] \
        [k_attrs][geo_region] = aggval_dict[k][geo_region] 

# print(json.dumps(json_loaded, indent=4))




In [10]:
# calc mix index
buff_dist_ft = params.mix_index_buffdist  # distance in feet--MIGHT NEED TO BE ADJUSTED FOR WGS 84--SEE OLD TOOL FOR HOW THIS WAS RESOLVED

out_dict = {}
for data_year in data_years:
    in_pcl_pt_fc = params.parcel_pt_fc_yr(in_year=data_year) # input fc of parcel data--must be points!
    year_dict = get_mix_idx(in_pcl_pt_fc, project_fc, ptype)
    out_dict[data_year] = year_dict

print(out_dict)

{2016: {'mix_index': 0.49111228436332865}, 2040: {'mix_index': 0.5444755587594678}}


In [28]:
def make_aggval_dict(aggval_df, metric_cols):
    aggdatadf_proj = aggval_df.loc[aggval_df[metname_col].isin(metric_cols)] \
                    [[metname_col, project_commtype, geo_region, k_year]]

    agg_dict_list = aggdatadf_proj.to_dict(orient='records')
    aggval_dict = {d[metname_col]:{project_commtype:d[project_commtype], 
                      geo_region:d[geo_region],
                      k_year:d[k_year]} for d in agg_dict_list}

    return aggval_dict

td = make_aggval_dict(df_agg, ['mix_index'])
td

{'mix_index': {'Arterials & Suburban Corridors': 0.4687834168845922,
  'Region': 0.9308534957530498,
  'year': 2040}}